<a href="https://colab.research.google.com/github/dan-a-iancu/airm/blob/master/Oro_Verde/Oro_Verde_Carbon_Sequestration_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**"Oro Verde" Carbon Offset Project**

This notebook implements a complete for the Oro Verde mini-case. The notebook assumes that you are  familiar with the context and have understood the basic optimization model.

In [1]:
#@title **Basic Setup.** We start by importing a few useful modules and reading the case data in.
#@markdown *Click on the "play" button on the left to run this entire cell.*

#@markdown - import/install modules
import numpy as np

import urllib
from urllib import request  # for file downloading

# Import pandas for data-frames
import pandas as pd
pd.options.display.max_rows = 15
pd.options.display.float_format = "{:,.2f}".format

# Make sure Matplotlib runs inline, for nice figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import matplotlib.ticker as ticker 

# install Gurobi (our linear optimization solver)
!pip install -i https://pypi.gurobi.com gurobipy
from gurobipy import *

# Ignore useless some warnings
import warnings
warnings.simplefilter(action="ignore")

print("Completed successfully!")

Looking in indexes: https://pypi.gurobi.com
     |████████████████████████████████| 11.5 MB 18.1 MB/s 
Completed successfully!


### Read in the case data and have a quick look at how it's organized

In [2]:
#@markdown - download the data as a CSV file (stored online)
url_Q1 = "https://raw.githubusercontent.com/dan-a-iancu/airm/master/Oro_Verde/Oro_Verde_data.csv"
local_file_Q1 = "Oro_Verde_data.csv"   # name of local file where you want to store the downloaded file
aux = urllib.request.urlretrieve(url_Q1, local_file_Q1)    # download from website and save it locally

#@markdown - read data into a pandas dataframe
mydata = pd.read_csv("Oro_Verde_data.csv", index_col = 0) 

print('Successfully read the data.')

#@markdown - print the dataframe with information on tree types
display(mydata)

#@markdown - set up any other problem parameters
land_avail = 150000              # available land area (in square feet)
water_avail = 50000              # available water (in gallons)
annual_seq_commit = 1800              # annual sequestration commitment
min_elms = 15   # minimum requirement on elm trees

#@markdown - create a list with all the types of trees (this will be very useful for creating our decision variables)
tree_types = list(mydata.index)

Successfully read the data.


,SequestrationRate,WaterRequirement,Width,SurvivalRate,SeedlingCost,SeedlingsAvailable
TreeType,,,,,,
Maple,4.30,76.00,130.00,0.70,5.40,"1,000.00"
Elm,3.70,48.00,"3,600.00",0.60,3.20,500.00
Spruce,2.80,34.00,400.00,0.80,4.50,"1,500.00"


# **Q1.**  

Suppose Oro Verde plants 500 maples, 500 elms, and 500 spruces. What would be the total cost, the total amount of water and land required, and the expected annual amount of carbon sequestration delivered in 10 years?


In [3]:
#@title Adjust the planting decisions and calculate and print relevant quantities

#@markdown - adjust how many trees of each type are planted
trees_to_plant = {}
for t in tree_types:
    trees_to_plant[t] = 500   # in Q1, we are planting 500 of each type

#@markdown - calculate and print the total cost, water and land required
cost = sum(trees_to_plant[t] * mydata["SeedlingCost"][t] for t in tree_types)
water_req = sum(trees_to_plant[t] * mydata["WaterRequirement"][t] for t in tree_types)
land_req = sum(trees_to_plant[t] * mydata["Width"][t] for t in tree_types)

print("Total cost: \t\t{:,.2f} ($)".format(cost))
print("Water required: \t{:,.2f} (gallons)".format(water_req))
print("Land required: \t\t{:,.2f} (square feet)".format(land_req))

Total cost: 		6,550.00 ($)
Water required: 	79,000.00 (gallons)
Land required: 		2,065,000.00 (square feet)


# **Q2.** 
Determine whether the plan in **Q1** is feasible.

*This section assumes that you have already run all the sections before, and particularly **Q1**. If that is not the case, please re-run everything above (e.g., by selecting this cell and choosing **Runtime > Run before**).*

Feasibility means that all the constraints are satisfied: 
 1. the land required does not exceed the land available
 2. the water required does not exceed the water available
 3. the seedlings used do not exceed the seedlings available
 4. the carbon sequestration commitment is met
 5. the number of Elm trees planted exceeds the minimum required

In [4]:
#@title Determine whether all constraints are satisfied
feasible = True # let's assume the proposal is feasible for now, and then check each constraint

# calculate the sequestration achieved
total_seq = sum(trees_to_plant[t] * mydata["SequestrationRate"][t] for t in tree_types)

# water
if (water_req > water_avail) :
    feasible = False
    print("The constraint for water is not satisfied: \n\t Required: {:,.2f}; Available: {:,.2f}" .\
          format(water_req,water_avail))

# land
if (land_req > land_avail) :
    feasible = False
    print("The constraint for land is not satisfied: \n\t Required: {:,.2f} ; Available: {:,.2f}".\
          format(land_req,land_avail))

# seedlings
for t in tree_types:
    if (trees_to_plant[t] > mydata["SeedlingsAvailable"][t]) :
        feasible = False
        print("The constraint for {} seedlings is not satisfied: \n\t Required: {:,.2f} ; Available: {:,.2f}".\
              format(t,water_req,water_avail))

# carbon sequestration
if (total_seq < annual_seq_commit) :
    feasible = False
    print("The constraint for carbon sequestration is not satisfied: \n\t Delivering: {:,.2f} ; Minimum required: {:,.2f}".
          format(total_seq,annual_seq_commit))

# elm min planting
if (trees_to_plant["Elm"] < min_elms) :
    feasible = False
    print("Too few elms planted. \n\t Planting: {:,.2f} ; Minimum required: {:,.2f}".\
          format(trees_to_plant["Elm"],min_elms))

if(feasible) :
    print("Solution is feasible!")
else:
    print("Solution is not feasible.")

The constraint for water is not satisfied: 
	 Required: 79,000.00; Available: 50,000.00
The constraint for land is not satisfied: 
	 Required: 2,065,000.00 ; Available: 150,000.00
Solution is not feasible.


# **Q3**
Set up an **Optimization Model** to determine the optimal number of trees of each type to plant. 

In [5]:
#@title Create the optimization model

#@markdown **Step 1.** Create an empty model
# the model is created with the function "Model" in the Gurobi package
# note that we give it a meaningful name
mymodel = Model("Oro Verde Tree Planting Model")

#@markdown **Step 2.** Create and add the decision variables
# we create the decisions: note that we only specify several arguments:
#   - name: is simply a string used internally, when displaying the model
#   - indices: this is a list of strings that represent the keys to a dictionary of variables
#             Here, we are making this the list 'tree_types', which means a decision will be created for each element of the list

trees_to_plant = mymodel.addVars(tree_types, name="trees_to_plant")

#@markdown **Step 3.** Calculate and add the objective in the model
#   - we use the function 'setOjective' in the Gurobi model
mymodel.setObjective( quicksum( trees_to_plant[t] * mydata["SeedlingCost"][t] for t in tree_types), GRB.MINIMIZE)

#@markdown **Step 4.** Calculate and add all the constraints
# we will name the constraints using the type of requirement

# water availability
mymodel.addConstr( quicksum( trees_to_plant[t] * mydata["WaterRequirement"][t] for t in tree_types) <= water_avail ,\
                  name = "water_avail" )

# land availability
mymodel.addConstr( quicksum( trees_to_plant[t] * mydata["Width"][t] for t in tree_types) <= land_avail, \
                  name = "land_avail" )

# carbon sequestration commitment - we save this in a variable for future use
seq_commit_constraint = mymodel.addConstr( quicksum( trees_to_plant[t] * mydata["SurvivalRate"][t] * mydata["SequestrationRate"][t] \
             for t in tree_types ) >= annual_seq_commit , name = "seq_commit" )

# minimum elm trees
mymodel.addConstr( trees_to_plant["Elm"] >= min_elms , name="minimum_Elm")

# seedling availability for each tree type
for t in tree_types :
    mymodel.addConstr( trees_to_plant[t] <= mydata["SeedlingsAvailable"][t], name="seedlings_" + t )

#@markdown **Step 5.** You can inspect the model, if you want (uncomment the next lines)
#mymodel.write("my_model.lp")    # write the model to a file with extension ".lp"
#f = open("my_model.lp", 'r')    # open a file handle
#print( f.read() )           # read the contents and print them
#f.close()              # close the file handle

#@markdown **Step 6.** Solve the model and display the status
# set the output to not be so verbose (it may confuse at first)
mymodel.Params.OutputFlag = 0
status=mymodel.optimize()

#@markdown **Step 7.** Print the optimal objective and optimal decisions
# print the value of the objective
costs_Q3 = mymodel.objVal
print("\n\nOptimal cost is: {:,.2f}".format(costs_Q3))

# print solution with a for loop
print("Optimal solution is to plant:")
for t in tree_types :
    print("\t{} : {:,.3f}".format(t,trees_to_plant[t].X))

Restricted license - for non-production use only - expires 2023-10-25


Optimal cost is: 3,213.10
Optimal solution is to plant:
	Maple : 582.798
	Elm : 20.621
	Spruce : 0.000
